In [1]:
import pandas as pd

In [20]:
import datetime as datetime

In [2]:
access_path = '../../data/processed/'
bal_file = 'bal_btw_rebal.dat'
# bal_file = 'balance_rebaldate_0801_all.dat'

#### 매일의 잔고(detail 테이블) INSERT

In [44]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [45]:
df_bal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-08-02,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,58030.0,116060.0,58030.0,116060.0,0.162358,NaN,NaN
1,2021-08-02,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101240.0,101240.0,101230.0,101230.0,0.141612,NaN,NaN
2,2021-08-02,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101140.0,202280.0,101145.0,202290.0,0.282986,NaN,NaN
3,2021-08-02,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104495.0,104495.0,104500.0,104500.0,0.146187,NaN,NaN
4,2021-08-02,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,109690.0,109690.0,109695.0,109695.0,0.153454,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2021-08-31,A237370,A009,적극투자형대규모,Alternative,KODEX 배당성장채권혼합,27.0,12165.0,328455.0,12130.0,327510.0,0.245409,NaN,NaN
2156,2021-08-31,A102780,A009,적극투자형대규모,Equity,KODEX 삼성그룹,17.0,10475.0,178075.0,10555.0,179435.0,0.134454,NaN,NaN
2157,2021-08-31,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,42485.0,127455.0,41985.0,125955.0,0.094380,NaN,NaN
2158,2021-08-31,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,58030.0,58030.0,58140.0,58140.0,0.043565,NaN,NaN


In [55]:
df_bal.date = df_bal['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal.date = df_bal['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [56]:
df_bal.date

0        8/2/2021 4:00:00 PM
1        8/2/2021 4:00:00 PM
2        8/2/2021 4:00:00 PM
3        8/2/2021 4:00:00 PM
4        8/2/2021 4:00:00 PM
                ...         
2155    8/31/2021 4:00:00 PM
2156    8/31/2021 4:00:00 PM
2157    8/31/2021 4:00:00 PM
2158    8/31/2021 4:00:00 PM
2159    8/31/2021 4:00:00 PM
Name: date, Length: 2160, dtype: object

In [57]:
from sqlalchemy import create_engine

# Defining our connection variables
username = 'kdyjpzwsyqrqpb' # replace with your username
password = '6b610da9417e361c2af3be1fd0c179061b5b258f0e6f7ff9e1da464ba221e46c' # replace with your password
ipaddress = 'ec2-35-174-35-242.compute-1.amazonaws.com' # change this to your db’s IP address is if not Panoply
port = 5432 # this is the standard port for Postgres, but change it to your port if needed
dbname = 'd2ke5vkurjkusr' # change this to...you guessed it...the name of your db

# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
          
# Create the connection
cnx = create_engine(postgres_str)

In [58]:
table_nm = 'detail'
df_bal.to_sql(table_nm, con=cnx, index=False, if_exists='append')

### 자산군별 비중(general 테이블) INSERT

In [59]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [60]:
df_bal_gby = df_bal.loc[:, ['date', 'userid', 'value', 'asset_class']].sort_values(
    ['date', 'userid'],
    ascending=True).groupby(['date', 'userid', 'asset_class'
                             ]).sum().reset_index(drop=False)

In [61]:
df_bal_gby = df_bal_gby.assign(wt=df_bal_gby.groupby(['date', 'userid'])['value'].transform(lambda x: x/x.sum()))

In [62]:
df_bal_gby.date = df_bal_gby['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal_gby.date = df_bal_gby['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [63]:
df_bal_gby

,date,userid,asset_class,value,wt
0,8/2/2021 4:00:00 PM,A001,Alternative,12150.0,0.016997
1,8/2/2021 4:00:00 PM,A001,Cash,6210.0,0.008687
2,8/2/2021 4:00:00 PM,A001,Equity,62705.0,0.087719
3,8/2/2021 4:00:00 PM,A001,Fixed Income,633775.0,0.886597
4,8/2/2021 4:00:00 PM,A002,Alternative,24300.0,0.024176
...,...,...,...,...,...
1075,8/31/2021 4:00:00 PM,A008,Fixed Income,58140.0,0.056539
1076,8/31/2021 4:00:00 PM,A009,Alternative,327510.0,0.245409
1077,8/31/2021 4:00:00 PM,A009,Cash,2785.0,0.002087
1078,8/31/2021 4:00:00 PM,A009,Equity,946115.0,0.708939


In [64]:
table_nm = 'general'
df_bal_gby.to_sql(table_nm, con=cnx, index=False, if_exists='append')